The files Data_train.csv and Data_test.csv contain data about cats.

In this task it is proposed to study the behaviour of wild and domestic cats based on several characteristics. There is some basic information about the cats (type, group). The cats had trainers. The trainer provides food for the cat, as well as trains some cats to complete an obstacle course (some do not have such training). Obstacle course performance was scored independently by three judges on a 100-point scale.

Column description:

* type – the type of the cat: wild or domestic.

* group - coded age group of the cat

* education - level of education of the trainer

* meal - type of the cat's diet

* preparation course - whether the cat has been trained in obstacle course (has had special training).

* score-1 - the first judge's score for the cat's obstacle course

* score-2 - the second judge’s score for the cat's obstacle course

* score-3 - the third judge's score for the cat's obstacle course

Further on, you can get a maximum of 4 points for the tasks.

Read the data into two pandas dataframes: df_t



---


Task 1 (0.25 points). Fill in the gaps in the column with a unique category (if the column with the gap is categorical), and with an average value (if the column is numeric). Fill in both df_train and df_test at the same time - in the same manner. In your answer, provide the number of different values required to fill in the gaps (it is equal to the number of new unique categories plus the number of average values to fill in the gaps in the numeric columns).



In [2]:
import pandas as pd
import numpy as np

In [25]:
df_train = pd.read_csv('Data_train.csv')
df_test = pd.read_csv('Data_test.csv')

In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   type                700 non-null    object
 1   group               680 non-null    object
 2   education           700 non-null    object
 3   meal                700 non-null    object
 4   preparation course  700 non-null    object
 5   score-1             700 non-null    int64 
 6   score-2             700 non-null    int64 
 7   score-3             700 non-null    int64 
dtypes: int64(3), object(5)
memory usage: 43.9+ KB


In [8]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   type                300 non-null    object
 1   group               288 non-null    object
 2   education           300 non-null    object
 3   meal                300 non-null    object
 4   preparation course  300 non-null    object
dtypes: object(5)
memory usage: 11.8+ KB


In [10]:
df_train['group'][2]

'group C'

In [17]:
L = [f'new_{x}' for x in range(1, 21)]

d = dict(enumerate(L, 1))

m = df_train.isna()
df2 = df_train.mask(m, m.cumsum().replace(d))
print (df2['group'].info())

<class 'pandas.core.series.Series'>
RangeIndex: 700 entries, 0 to 699
Series name: group
Non-Null Count  Dtype 
--------------  ----- 
700 non-null    object
dtypes: object(1)
memory usage: 5.6+ KB
None


In [26]:
df_train['group'].fillna('unknown')

,group
0,group B
1,group C
2,group C
3,group B
4,group E
...,...
695,group C
696,group B
697,group E
698,group E


In [27]:
df_test['group'].fillna('unknown')

,group
0,group D
1,group C
2,group B
3,group A
4,group B
...,...
295,group B
296,group C
297,group D
298,group C


In [28]:
combined = pd.concat([df_train, df_test], axis=0)

categorical_cols = combined.select_dtypes(include='object').columns
numeric_cols = combined.select_dtypes(include='number').columns

num_categorical_fillers = sum(combined[col].isnull().any() for col in categorical_cols)

num_numeric_fillers = sum(combined[col].isnull().any() for col in numeric_cols)

total_fill_values = num_categorical_fillers + num_numeric_fillers

print("Number of different values used to fill the gaps:", total_fill_values)

Number of different values used to fill the gaps: 4


Task 2 (0.3 points). The judge decides that the cat has passed the obstacle course if they give it more than 50 points. The cat is considered to have passed the obstacle course if all judges gave it more than 50 points. In df_train, create a 'Pass' column and write 1 if the cat passed the obstacle course and 0 otherwise. In your answer, record how many cats from df_train did not pass the obstacle course.

In df_test, the information about the judges' scores is hidden from you, so you don't know if the cat passed the obstacle course or not - this is what you will have to predict in the tasks below.



In [29]:
# df_train['Pass'] = df_train.apply(lambda row: 1 if row['score-1'] > 50 and row['score-2'] and row['score-3'] > 50 else 0)


In [30]:
def categorize(row):
    if row['score-1'] > 50 and row['score-2'] > 50 and row['score-3'] > 50:
        return 1
    else:
        return 0

df_train['Pass'] = df_train.apply(categorize, axis=1)

In [36]:
conditions = [
    (df_train['score-1'] > 50) & (df_train['score-2'] > 50) & (df_train['score-3'] > 50),
    ~((df_train['score-1'] > 50) & (df_train['score-2'] > 50) & (df_train['score-3'] > 50))
]
choices = [1, 0]
df_train['Pass2'] = np.select(conditions, choices, default=0)

In [37]:
df_train['Pass'].nunique()

2

In [38]:
df_train['Pass'].value_counts()

,count
Pass,
1,555
0,145


In [39]:
df_train['Pass'].value_counts(normalize=True) * 100

,proportion
Pass,
1,79.285714
0,20.714286


In [40]:
df_train['Pass'].value_counts().to_frame(name='count') \
    .assign(percentage=lambda x: 100 * x['count'] / x['count'].sum())

,count,percentage
Pass,,
1,555,79.285714
0,145,20.714286


Task 3 (each point is 0.25 points, 1.25 points maximum).

Complete this task using df_train data.

1) Among all wild cats, find the proportion of cats that passed the obstacle course. Calculate the same proportion for domestic cats. In your answer, give the modulus of the difference of these fractions. Round your answer to hundredths.

In [41]:
wild_cats = df_train[df_train['type'] == 'wild']
domestic_cats = df_train[df_train['type'] == 'domestic']

In [59]:
wild_passed = (wild_cats['Pass'].value_counts(normalize=True))[1]

In [60]:
domestic_passed = (domestic_cats['Pass'].value_counts(normalize=True))[1]

In [61]:
print(np.abs(wild_passed - domestic_passed))

0.022580460755730947


2) How many cats among those who did not pass the obstacle course had trainers with a "high school" level of education?



In [62]:
failed_cats = df_train[df_train['Pass'] == 0]

In [63]:
failed_cats['education'].value_counts()

,count
education,
some high school,42
high school,35
some college,28
associate's degree,22
bachelor's degree,12
master's degree,6


3) How many wild cats among those who have passed the obstacle course have not had special training course?



In [77]:
passed_cats = df_train[df_train['Pass'] == 1]

In [ ]:
df_train['preparation course'].value_counts()

In [78]:
# passed_cats[passed_cats['type'] == 'wild'][passed_cats['preparation course'] == 'none'].count()
passed_cats[passed_cats['type'] == 'wild']['preparation course'].value_counts()

,count
preparation course,
none,152
completed,116


4) What is the median of the scores given by the first judge?



In [74]:
df_train['score-1'].median()

66.0

5) Find the interquartile range of the third judge's score (third quartile minus first quartile) for domestic cats that have not received special training.

Comment: To calculate the quartiles of a discrete distribution, use lower interpolation. This means that if the quartile you are looking for lies between the two dimensions i and j, the quartile value is i.



In [86]:
scores = df_train[(df_train['type'] == 'domestic') &
                  (df_train['preparation course'] == 'none')]['score-3'].values

q1 = np.percentile(scores, 25, method='lower')  # or interpolation='lower'
q3 = np.percentile(scores, 75, method='lower')

iqr = q3 - q1
print("IQR:", iqr)

IQR: 20


In [87]:
q1 = df_train['score-3'].quantile(0.25, interpolation='lower')
q3 = df_train['score-3'].quantile(0.75, interpolation='lower')

print("IQR:", q3 - q1)

IQR: 21


Task 4 (0.7 points).

a) (0.3 points). Further on, use only categorical columns. Encode them using One-hot encoding taking into account that we do not want to get multicollinearity in the new data. How many numeric columns did we get from the original categorical columns? Encode both df_train and df_test.




In [101]:
categorical_cols = df_train.select_dtypes(include='object').columns
combined_encoded = pd.get_dummies(df_train, columns=categorical_cols, drop_first=True)
combined_encoded = combined_encoded.drop(['Pass2', 'Pass'], axis=1)
combined_encoded.head()

,score-1,score-2,score-3,type_wild,group_group B,group_group C,group_group D,group_group E,education_bachelor's degree,education_high school,education_master's degree,education_some college,education_some high school,meal_standard,preparation course_none
0,63,67,67,True,True,False,False,False,False,False,False,False,True,True,False
1,67,69,75,False,False,True,False,False,True,False,False,False,False,True,True
2,69,90,88,False,False,True,False,False,False,False,False,True,False,True,False
3,62,64,66,False,True,False,False,False,False,False,False,False,True,True,True
4,93,90,83,True,False,False,False,True,False,False,False,True,False,False,True


In [102]:
combined_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype
---  ------                       --------------  -----
 0   score-1                      700 non-null    int64
 1   score-2                      700 non-null    int64
 2   score-3                      700 non-null    int64
 3   type_wild                    700 non-null    bool 
 4   group_group B                700 non-null    bool 
 5   group_group C                700 non-null    bool 
 6   group_group D                700 non-null    bool 
 7   group_group E                700 non-null    bool 
 8   education_bachelor's degree  700 non-null    bool 
 9   education_high school        700 non-null    bool 
 10  education_master's degree    700 non-null    bool 
 11  education_some college       700 non-null    bool 
 12  education_some high school   700 non-null    bool 
 13  meal_standard                700 non-null    bool 

In [103]:
num_new_numeric_cols = len(combined_encoded.columns) - len(numeric_cols)

print(f"Number of new numeric columns from one-hot encoding: {num_new_numeric_cols}")

Number of new numeric columns from one-hot encoding: 12


(а если не убирать Pass, то 13)